In [36]:
import pandas as pd
import sqlite3
import geopandas as gpd
from shapely.geometry import Point
import folium


### Query for data

In [37]:

# Create a connection to the SQLite database
conn = sqlite3.connect('transit_database.db')

# SQL query
query = """
SELECT stations.*, 
       station_entrances."entrance_id", 
       entrances."longitude" as entrance_longitude, 
       entrances."latitude" as entrance_latitude
FROM stations
LEFT JOIN station_entrances
ON stations."station_id" = station_entrances."station_id"
LEFT JOIN entrances
ON station_entrances."entrance_id" = entrances."entrance_id"
"""

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Don't forget to close the connection
conn.close()


### Create Geopandas and geometry objects

In [38]:
gdf = gpd.GeoDataFrame({
    'geometry': [Point(1, 1), Point(2, 2)],
    'entrance_node': [Point(1, 1), Point(2, 2)]
})

### For station coordinates
# Ensure 'Longitude' and 'Latitude' are numeric
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

# Drop any rows with NaN in 'Longitude' or 'Latitude'
df = df.dropna(subset=['longitude', 'latitude'])

# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
df = df.drop(['longitude', 'latitude'], axis=1)

gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf['entrance_node'] = ''

### For entrances coordinates
# Ensure 'Longitude' and 'Latitude' are numeric
df['entrance_longitude'] = pd.to_numeric(df['entrance_longitude'], errors='coerce')
df['entrance_latitude'] = pd.to_numeric(df['entrance_latitude'], errors='coerce')

# Drop any rows with NaN in 'Longitude' or 'Latitude'
df = df.dropna(subset=['entrance_longitude', 'entrance_latitude'])

# Convert DataFrame to GeoDataFrame
entrance_node = [Point(xy) for xy in zip(df['entrance_longitude'], df['entrance_latitude'])]
df = df.drop(['entrance_longitude', 'entrance_latitude'], axis=1)

gdf['entrance_node'] = entrance_node


ValueError: Length of values (225) does not match length of index (657)

In [14]:
gdf = gpd.GeoDataFrame({
    'geometry': [Point(1, 1), Point(2, 2)],
    'entrance_node': [Point(1, 1), Point(2, 2)]
})

gdf

,geometry,entrance_node
0,POINT (1.00000 1.00000),POINT (1 1)
1,POINT (2.00000 2.00000),POINT (2 2)


In [4]:
gdf

,name,station_id,service_provider_name,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,entrance_id,geometry
9,Kajang,KB06,Keretapi Tanah Melayu,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,7.702629e+09,POINT (101.79089 2.98382)
10,Kajang,KB06,Keretapi Tanah Melayu,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,1.022384e+10,POINT (101.79008 2.98298)
61,Sentul Timur,AG1,Rapid KL,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,None,None,None,9.913027e+09,POINT (101.69538 3.18561)
62,Pudu,AG10,Rapid KL,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,None,None,None,1.079685e+10,POINT (101.71214 3.13476)
72,Titiwangsa,AG3,Rapid KL,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,None,None,None,5.386931e+09,POINT (101.69543 3.17252)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,Putra Permai,PY37,Rapid KL,PY,Putrajaya Line,12,Yellow,#facd04,Klang Valley,None,None,None,1.026624e+10,POINT (101.66067 2.98358)
355,Putra Permai,PY37,Rapid KL,PY,Putrajaya Line,12,Yellow,#facd04,Klang Valley,None,None,None,1.061168e+10,POINT (101.66123 2.98331)
356,16 Sierra,PY38,Rapid KL,PY,Putrajaya Line,12,Yellow,#facd04,Klang Valley,None,None,None,1.061168e+10,POINT (101.65456 2.96502)
357,Cyberjaya Utara,PY39,Rapid KL,PY,Putrajaya Line,12,Yellow,#facd04,Klang Valley,None,None,None,1.084000e+10,POINT (101.65721 2.94971)


In [ ]:
mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=11)


In [ ]:
def toMapORS(data,line,params_iso,client):
    # Set up folium map
    if not line in data.values:
        print('{} is not in data frame'.format(line))
        temp = data['Route Name'].unique()
        print('Choose from the following: ')
        print(temp)
        return
    if line != None:
        data = data[data['Route Name']==line]
    starting_location = (data['Latitude'].iloc[0],data['Longitude'].iloc[0])
    mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=11)

    isoGeoJsonRetriever(params_iso,data,client)
    isoVisualizer(mapped,data)
    return mapped,data

 style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' : 0.35 if x['properties']['value']<400 else (0.25 if 400.0<x['properties']['value']<700.0 else 0.05),
                                'weight':2,
                                'fillColor' :'#4ef500' if x['properties']['value']<400 else ('#2100f5' if 400.0<x['properties']['value']<700.0 else '#f50000')}
                                                    #('#6234eb' if x['properties']['value']==600.0 else '#6234eb')

    #necessary to create 'locations' column to know specific coordinates for marking
    stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)
    
    for index, row in stations.iterrows():
        folium.features.GeoJson(row['iso'],style_function = style_function).add_to(maps) # Add GeoJson to map
        if map_icon!="":
            if row['Colour Hex Code']!='':
                station_color = row['Colour Hex Code']
            else:
                station_color =icon_color
            folium.map.Marker(list(reversed(row['locations'])), # reverse coords due to weird folium lat/lon syntax
                                icon=folium.Icon(color='lightgray',
                                            icon_color=station_color,
                                            icon=map_icon,
                                            prefix='fa',
                                                ),
                            popup=row['Name'],
                            ).add_to(maps) # Add apartment locations to map      
    print("Done!")

#Perform isochrone request and generates a ne